In [57]:
import requests
from bs4 import BeautifulSoup
from time import sleep
from tqdm.notebook import tqdm
import csv

In [4]:
url_template = 'https://www.aclweb.org/anthology/2020.emnlp-main.%d/'

In [7]:
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

In [55]:
data = []
for i in tqdm(range(1,752)):
    try:
        r = requests.get(url_template % i, headers=headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        title = soup.find('meta', {'name': 'citation_title'})['content']
        abstract = soup.find('div', {'class':'acl-abstract'}).text[8:]
        author_list = soup.find_all('meta', {'name':'citation_author'})
        authors = [author['content'] for author in author_list]
        pdf_url = soup.find('meta', {'name': 'citation_pdf_url'})['content']

        data.append({
            'title': title,
            'abstract': abstract,
            'authors': authors,
            'pdf_url': pdf_url,
            'url' : r.url
        })

        sleep(0.2)
    except Exception as e:
        print(e)

In [59]:
with open('../data/EMNLP2020.csv', 'w', newline='', encoding="utf-8") as f:
    dict_writer = csv.DictWriter(f, data[0].keys())
    dict_writer.writeheader()
    dict_writer.writerows(data)